In [7]:
!pip install -q --upgrade "transformers==4.40.0" "datasets==2.18.0" "accelerate==0.29.3" "evaluate==0.4.1" "bitsandbytes==0.43.1" "huggingface_hub==0.22.2" "trl==0.8.6" "peft==0.10.0"

In [ ]:
%%writefile llama_3_7b_fsdp_qlora.yaml
# script parameters
model_id: "/root/llama/models/models--unsloth--llama-3-8b/snapshots/a299e1185075f847b3f9d5434713852093e23684"
dataset_path: "."                      # path to dataset
max_seq_len:  3072 # 2048              # max sequence length for model and packing of the dataset
# training parameters
output_dir: "./llama-3-70b-hf-no-robot" # Temporary output directory for model checkpoints
report_to: "tensorboard"               # report metrics to tensorboard
learning_rate: 0.0002                  # learning rate 2e-4
lr_scheduler_type: "constant"          # learning rate scheduler
num_train_epochs: 3                    # number of training epochs
per_device_train_batch_size: 1         # batch size per device during training
per_device_eval_batch_size: 1          # batch size for evaluation
gradient_accumulation_steps: 2         # number of steps before performing a backward/update pass
optim: adamw_torch                     # use torch adamw optimizer
logging_steps: 10                      # log every 10 steps
save_strategy: epoch                   # save checkpoint every epoch
evaluation_strategy: epoch             # evaluate every epoch
max_grad_norm: 0.3                     # max gradient norm
warmup_ratio: 0.03                     # warmup ratio
bf16: true                             # use bfloat16 precision
tf32: true                             # use tf32 precision
gradient_checkpointing: true           # use gradient checkpointing to save memory
# FSDP parameters: https://huggingface.co/docs/transformers/main/en/fsdp
fsdp: "full_shard auto_wrap offload" # remove offload if enough GPU memory
fsdp_config:
  backward_prefetch: "backward_pre"
  forward_prefetch: "false"
  use_orig_params: "false"

In [7]:
# !git clone https://github.com/meta-llama/llama-recipes.git

Cloning into 'llama-recipes'...
remote: Enumerating objects: 5540, done.
remote: Counting objects: 100% (3502/3502), done.
remote: Compressing objects: 100% (815/815), done.
remote: Total 5540 (delta 2829), reused 3085 (delta 2623), pack-reused 2038
Receiving objects: 100% (5540/5540), 21.45 MiB | 33.59 MiB/s, done.
Resolving deltas: 100% (3430/3430), done.
Updating files: 100% (207/207), done.


In [9]:
mode_id = "/root/llama/models/models--unsloth--llama-3-8b/snapshots/a299e1185075f847b3f9d5434713852093e23684/"


In [4]:
!torchrun --nnodes 1 --nproc_per_node 4 llama-recipes/recipes/finetuning/finetuning.py --dataset "custom_dataset" --custom_dataset.file "local_dataset.py" --enable_fsdp --low_cpu_fsdp --pure_bf16 --model_name /root/llama/models/models--unsloth--llama-3-8b/snapshots/a299e1185075f847b3f9d5434713852093e23684 --batch_size_training 1 --dist_checkpoint_root_folder model_checkpoints --dist_checkpoint_folder fine-tuned


[2024-05-12 18:14:22,194] torch.distributed.run: [WARNING] 
[2024-05-12 18:14:22,194] torch.distributed.run: [WARNING] *****************************************
[2024-05-12 18:14:22,194] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-05-12 18:14:22,194] torch.distributed.run: [WARNING] *****************************************
Clearing GPU cache for all ranks
--> Running with torch dist debug set to detail
Traceback (most recent call last):
  File "/root/llama/llama-recipes/recipes/finetuning/finetuning.py", line 8, in <module>
    fire.Fire(main)
  File "/opt/conda/lib/python3.10/site-packages/fire/core.py", line 143, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/opt/conda/lib/python3.10/site-packages/fire/core.py", line 477, in _Fire
   